# Análisis Exploratorio de Datos (EDA)
## 1. Carga y Fusión de Datos

Primero, cargaremos los cuatro conjuntos de datos y los uniremos en un único DataFrame utilizando la columna customerID como clave.

In [2]:
import pandas as pd
import io

try:
    df_contract = pd.read_csv('contract.csv')
    df_internet = pd.read_csv('internet.csv')
    df_personal = pd.read_csv('personal.csv')
    df_phone = pd.read_csv('phone.csv')

    # Fusionar los DataFrames
    df = pd.merge(df_contract, df_personal, on='customerID', how='outer')
    df = pd.merge(df, df_internet, on='customerID', how='outer')
    df = pd.merge(df, df_phone, on='customerID', how='outer')

    print("Datos cargados y fusionados correctamente.")
    print("Forma del DataFrame final:", df.shape)
    display(df.head())

except Exception as e:
    print(f"Ocurrió un error al cargar los datos: {e}")

Datos cargados y fusionados correctamente.
Forma del DataFrame final: (7043, 20)


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,Female,0,Yes,No,DSL,No,Yes,No,No,No,No,NaN
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,Male,0,No,No,DSL,Yes,No,Yes,No,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,No,No,DSL,Yes,Yes,No,No,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,Male,0,No,No,DSL,Yes,No,Yes,Yes,No,No,NaN
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,No,No,Fiber optic,No,No,No,No,No,No,No


## 2. Inspección Inicial y Limpieza
Revisaremos la estructura general, los tipos de datos y los valores ausentes.

In [3]:
# Obtener información general del DataFrame
print("Información del DataFrame:")
df.info()

# Revisar valores ausentes
print("\nValores ausentes por columna:")
print(df.isnull().sum())

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   gender            7043 non-null   object 
 9   SeniorCitizen     7043 non-null   int64  
 10  Partner           7043 non-null   object 
 11  Dependents        7043 non-null   object 
 12  InternetService   5517 non-null   object 
 13  OnlineSecurity    5517 non-null   object 
 14  OnlineBackup      5517 non-null   object 
 15  DeviceProtection  5517 non-null   object 
 16  TechSupport    

### Observaciones clave:

- BeginDate y EndDate son de tipo object y deberían ser fechas.
- TotalCharges es de tipo object y tiene 11 valores nulos. Probablemente sean clientes nuevos sin cargos totales aún.
- Las columnas de servicios de internet (InternetService, OnlineSecurity, etc.) tienen 1526 valores nulos. Esto probablemente corresponde a clientes que no tienen contratado el servicio de internet.
- La columna MultipleLines tiene 682 valores nulos. Esto corresponde a clientes que no tienen servicio telefónico.

## 3. Creación de la Característica Objetivo

Crearemos nuestra variable objetivo Churn (cancelación) a partir de la columna EndDate. Si EndDate es 'No', el cliente no ha cancelado (0). Si tiene una fecha, sí lo ha hecho (1).

In [4]:
# Crear la columna objetivo 'Churn'
df['Churn'] = (df['EndDate'] != 'No').astype(int)

# Comprobar la distribución de la nueva columna
print("Distribución de la cancelación de clientes:")
print(df['Churn'].value_counts(normalize=True))

Distribución de la cancelación de clientes:
0    0.73463
1    0.26537
Name: Churn, dtype: float64


Observación: Aproximadamente el 26.5% de los clientes en este conjunto de datos ha cancelado el servicio. Es un desequilibrio de clases moderado que deberemos tener en cuenta en el modelado.

## 4. Preguntas Aclaratorias para el Líder de Equipo

1. Manejo de Nulos: Los valores nulos en TotalCharges (11 casos) parecen ser de clientes muy recientes. ¿Debemos eliminarlos o imputarles un valor (por ejemplo, el de MonthlyCharges)?
2. Fecha de los Datos: El enunciado indica que "La información del contrato es válida a partir del 1 de febrero de 2020". ¿Significa que este es un snapshot y no debemos preocuparnos por la evolución en el tiempo de los contratos? Esto es clave para decidir cómo usar las fechas BeginDate y EndDate.
3. Correlación de Servicios: Los clientes sin servicio de internet tienen valores nulos en todas las columnas relacionadas (OnlineSecurity, TechSupport, etc.). ¿Es correcto asumir que estos nulos significan "No contratado" y rellenarlos con un 'No' o una categoría similar? Lo mismo aplicaría para los nulos en MultipleLines para quienes no tienen servicio telefónico.
4. Definición de 'Cliente Activo': ¿Consideramos "activos" a todos los clientes cuyo EndDate es 'No' al 1 de febrero de 2020? Esto nos ayudará a definir correctamente la población sobre la que queremos predecir.

## 5. Plan Aproximado de Resolución
#### Preprocesamiento y Limpieza de Datos:

Convertir las columnas de fechas (BeginDate, EndDate) a formato datetime. Crear una nueva característica como la duración del contrato (tenure).

Manejar los valores nulos identificados en TotalCharges y en las columnas de servicios de internet y telefonía, basándonos en las respuestas a las preguntas aclaratorias. Convertir TotalCharges a tipo numérico.

#### Ingeniería y Selección de Características (Feature Engineering):

Convertir todas las variables categóricas a un formato numérico que los modelos puedan entender, utilizando técnicas como One-Hot Encoding o Label Encoding según corresponda.

Analizar la correlación entre las características y la variable objetivo para entender cuáles son más influyentes y potencialmente descartar las que no aporten información.

#### Entrenamiento y Evaluación de Modelos:

Dividir el conjunto de datos en entrenamiento y prueba.

Entrenar al menos dos modelos de clasificación distintos (por ejemplo, Regresión Logística como línea base y un modelo más complejo como LightGBM o Random Forest) para comparar su rendimiento.

Evaluar los modelos utilizando la métrica principal AUC-ROC y la secundaria de exactitud (Accuracy). Se prestará especial atención al desequilibrio de clases, posiblemente usando técnicas como el sobremuestreo (SMOTE) o ajustando los pesos de las clases en los modelos.

#### Ajuste Fino y Selección del Modelo Final:

Realizar una búsqueda de hiperparámetros (por ejemplo, con GridSearchCV o RandomizedSearchCV) sobre el modelo con mejor rendimiento para optimizarlo.

Seleccionar el modelo final basándose en su rendimiento en el conjunto de prueba y su interpretabilidad, asegurando que cumpla o supere el umbral de AUC-ROC >= 0.88 para la máxima puntuación.